# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# OpenPose Installation and Save

## Path Settings

In [2]:
import os
OPENPOSE_PATH="./openpose/"
HOME_PATH='./'

## Installing OpenPose

In [3]:
#@title
from os.path import exists, join, basename, splitext

# see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
# install new CMake becaue of CUDA10
!wget -q https://cmake.org/files/v3.17/cmake-3.17.2-Linux-x86_64.tar.gz
!tar xfz cmake-3.17.2-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

# install system dependencies
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
!rm -rf openpose
# clone openpose
!git clone -q --depth 1 $git_repo_url
# --recursive necessary in the line below, as otherwise you can (sometimes) get "lpthreads" errors in cmake ("undefined reference to `pthread_create'" etc). See, for example, https://github.com/facebookarchive/caffe2/issues/1234
!sed -i 's/execute_process(COMMAND git checkout --recursive master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
!cd openpose && git submodule update --init --recursive --remote

!wget "https://drive.usercontent.google.com/download?id=1QCSxJZpnWvM00hx49CJ2zky7PWGzpcEh&export=download&authuser=0&confirm=t&uuid=50355e54-0445-4875-9d0e-3866a1f6239d&at=APZUnTVA3sHsK5rk-u2O6phIms2L%3A1711987634035" -O models.zip

!unzip models.zip -d openpose

Selecting previously unselected package liblmdb0:amd64.
(Reading database ... 121913 files and directories currently installed.)
Preparing to unpack .../00-liblmdb0_0.9.24-1build2_amd64.deb ...
Unpacking liblmdb0:amd64 (0.9.24-1build2) ...
Selecting previously unselected package libgflags2.2.
Preparing to unpack .../01-libgflags2.2_2.2.2-2_amd64.deb ...
Unpacking libgflags2.2 (2.2.2-2) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../02-libgflags-dev_2.2.2-2_amd64.deb ...
Unpacking libgflags-dev (2.2.2-2) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../03-libgoogle-glog0v5_0.5.0+really0.4.0-2_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.5.0+really0.4.0-2) ...
Selecting previously unselected package libunwind-dev:amd64.
Preparing to unpack .../04-libunwind-dev_1.3.2-2build2.1_amd64.deb ...
Unpacking libunwind-dev:amd64 (1.3.2-2build2.1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to un

In [4]:
!sed -i 's/executeShInItsFolder "getModels.sh"/# executeShInItsFolder "getModels.sh"/g' ./openpose/scripts/ubuntu/install_openpose_JetsonTX2_JetPack3.1.sh
!sed -i 's/executeShInItsFolder "getModels.sh"/# executeShInItsFolder "getModels.sh"/g' ./openpose/scripts/ubuntu/install_openpose_JetsonTX2_JetPack3.3.sh
!sed -i 's/download_model("BODY_25"/# download_model("BODY_25"/g' ./openpose/CMakeLists.txt
!sed -i 's/78287B57CF85FA89C03F1393D368E5B7/# 78287B57CF85FA89C03F1393D368E5B7/g' ./openpose/CMakeLists.txt
!sed -i 's/download_model("body (COCO)"/# download_model("body (COCO)"/g' ./openpose/CMakeLists.txt
!sed -i 's/5156d31f670511fce9b4e28b403f2939/# 5156d31f670511fce9b4e28b403f2939/g' ./openpose/CMakeLists.txt
!sed -i 's/download_model("body (MPI)"/# download_model("body (MPI)"/g' ./openpose/CMakeLists.txt
!sed -i 's/2ca0990c7562bd7ae03f3f54afa96e00/# 2ca0990c7562bd7ae03f3f54afa96e00/g' ./openpose/CMakeLists.txt
!sed -i 's/download_model("face"/# download_model("face"/g' ./openpose/CMakeLists.txt
!sed -i 's/e747180d728fa4e4418c465828384333/# e747180d728fa4e4418c465828384333/g' ./openpose/CMakeLists.txt
!sed -i 's/download_model("hand"/# download_model("hand"/g' ./openpose/CMakeLists.txt
!sed -i 's/a82cfc3fea7c62f159e11bd3674c1531/# a82cfc3fea7c62f159e11bd3674c1531/g' ./openpose/CMakeLists.txt

## Make (takes \~ 10 minutes!)

In [5]:
!cd openpose && rm -rf build || true && mkdir build

In [6]:
cmake_file='/content/openpose/CMakeLists.txt'
!cd openpose && sed -i 's/-DBUILD_python=OFF/-DBUILD_python=ON/g' $cmake_file
!cd openpose && sed -i 's/-DBUILD_python_layer=OFF/-DBUILD_python_layer=ON/g' $cmake_file
!cd openpose && sed -i 's/option(BUILD_PYTHON "Build OpenPose python." OFF)/option(BUILD_PYTHON "OpenPose python." ON)\noption(BUILD_BIN_FOLDER "Copy 3rd-party DLL files." ON)/g' $cmake_file

In [7]:
!cd openpose && cd build && cmake .. -DUSE_CUDNN=OFF -DGENERATE_PYTHON_BINDINGS:BOOL="1" -DPYTHON_LIBRARY='/usr/lib/x86_64-linux-gnu/libpython3.10.so' && make -j`nproc`

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc - works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ - works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- GCC detected, adding compile flags
-- GCC detected, adding compile flags
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE  
-- Found CUDA: /usr/local/cuda (found version "12.2") 
-- Building with CUDA.
-- CUDA detected: 12.2
-- Added CUDA NVCC flags for: sm_75
-- Found cuDNN

# Install YoloV8 / Ultralytics
Most of the Yolo interface code in this notebook is from the official YOLOv8 [tutorial](https://colab.research.google.com/github/roboflow-ai/notebooks/blob/main/notebooks/train-yolov8-object-detection-on-custom-dataset.ipynb#scrollTo=tdSMcABDNKW-).

In [8]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.7/78.2 GB disk)


In [9]:
import cv2
import numpy as np
from ultralytics import YOLO
from IPython.display import display, Image
from google.colab.patches import cv2_imshow

model = YOLO('yolov8m.pt')

# Auxilliary Variables / Folders, Extract frames at 30 fps

In [10]:
final_keypoints_path = "/content/final_keypoints/"
manual_crop_videos_path = "/content/manual_crop_videos/"
manual_crop_keypoints_path = "/content/manual_crop_keypoints/"
bound_and_crop_videos_path = "/content/bound_and_crop_videos/"
bound_and_crop_frames_path = "/content/bound_and_crop_frames/"
bound_and_crop_keypoints_path = "/content/bound_and_crop_keypoints/"
for path in [final_keypoints_path, manual_crop_videos_path, manual_crop_keypoints_path, bound_and_crop_videos_path, bound_and_crop_frames_path, bound_and_crop_keypoints_path]:
  os.makedirs(path)

NORMALIZED_HEIGHT = 800

FileExistsError: [Errno 17] File exists: '/content/final_keypoints/'

## Extract frames from all videos

In [13]:
frames_path = "/content/frames/"
os.mkdir(frames_path)

FileExistsError: [Errno 17] File exists: '/content/frames/'

In [14]:
import subprocess
from tqdm import tqdm
raw_videos_ver2_path = "/content/drive/MyDrive/ataxia_dataset/raw_videos_ver2/"
# Extract frames for all videos & save them
for video in os.listdir(raw_videos_ver2_path):
  if ".mov" not in video:
    continue
  else:
    # Break the video into frames at 30 fps
    if os.path.exists(os.path.join(frames_path, video[:-4])):
      continue
    os.mkdir(os.path.join(frames_path, video[:-4]))
    command = ["ffmpeg", "-i", os.path.join(raw_videos_ver2_path, video), "-vf", "fps=30", os.path.join(frames_path, video[:-4], "output_%05d.jpg")]
    out = subprocess.run(command, stderr=subprocess.PIPE)

# `cur_video_num` variable that changes after every pass

In [82]:
cur_video_num = 16

# Get Keypoints for patient

## 1, Pass through OpenPose directly and visualize the output

In [83]:
stored = False

In [84]:
# Extract keypoints using openpose
if not os.path.exists(os.path.join(manual_crop_keypoints_path, str(cur_video_num))):
  os.mkdir(os.path.join(manual_crop_keypoints_path, str(cur_video_num)))
  openposed = ["./build/examples/openpose/openpose.bin", \
                "--image_dir", os.path.join(frames_path, str(cur_video_num)), \
                "--number_people_max", "1", \
                "--write_json", os.path.join(manual_crop_keypoints_path, str(cur_video_num)), \
                "--write_video", os.path.join(manual_crop_videos_path, str(cur_video_num) + ".mp4"), \
                "--write_video_fps", "30", \
                "--display", "0", \
                "--keypoint_scale", "3", \
                "--model_pose", "BODY_25"]
  openpoze = subprocess.run(openposed, stderr=subprocess.PIPE, stdout=subprocess.PIPE, cwd="/content/openpose") # Need to execute this from the openpose directory, thus `cwd` is specified
  print(openpoze.stderr)

b"ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers\n  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)\n  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --

In [85]:
good_or_bad = input("Is the output okay? (y/n) : ").lower()
temp_kpt_path = os.path.join(manual_crop_keypoints_path, str(cur_video_num))
final_kpt_path = os.path.join(final_keypoints_path, str(cur_video_num))
if good_or_bad == "y" and stored == False:
  import shutil
  shutil.copytree(temp_kpt_path, final_kpt_path)
  stored = True

Is the output okay? (y/n) : y


In [ ]:
if stored and cur_video_num % 10 == 0:
  !zip final.zip final_keypoints/
  shutil.copy(final.zip, "/content/drive/MyDrive/ataxia_dataset/")

## 2, If problems, pass through YoloV8 to bound-and-crop and then do a OpenPose extraction again

In [19]:
if not stored:
  for frame in os.listdir(os.path.join(frames_path, str(cur_video_num))):
    print(os.path.join(bound_and_crop_frames_path, str(cur_video_num), frame))
    if os.path.exists(os.path.join(bound_and_crop_frames_path, str(cur_video_num), frame)):
      continue
    # Define input image
    image = cv2.imread(os.path.join(frames_path, str(cur_video_num), frame))

    # Get image dimensions
    (height, width) = image.shape[:2]

    # Get bounding boxes around objects
    results = model.predict(source=image, iou=0.7, verbose=False)

    # Initialize the final placeholders
    max_conf = -1
    max_conf_person = None

    # Loop over the detections
    for result in results:
      for (xywh, confidence, cls) in zip(result.boxes.xywh, result.boxes.conf, result.boxes.cls):

        if cls == 0 and confidence > max_conf:
            # Object detected
            center_x = int(xywh[0]) # * width
            center_y = int(xywh[1]) # * height
            w = int(xywh[2]) # * width
            h = int(xywh[3]) # * height

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            # Add the detection to the list of people
            max_conf_person = (x, y, w, h)
            max_conf = confidence

    x, y, w, h = max_conf_person
    cropped_image = image[y:y+h, x:x+w]
    aspect_ratio = w/h
    new_width = int(NORMALIZED_HEIGHT * aspect_ratio)

    # Resize the image
    resized_image = cv2.resize(cropped_image, (new_width, NORMALIZED_HEIGHT))

    if not os.path.exists(os.path.join(bound_and_crop_frames_path, str(cur_video_num))):
      os.mkdir(os.path.join(bound_and_crop_frames_path, str(cur_video_num)))
    cv2.imwrite(os.path.join(bound_and_crop_frames_path, str(cur_video_num), frame), resized_image)

In [20]:
if not stored:
  os.mkdir(os.path.join(bound_and_crop_keypoints_path, str(cur_video_num)))
  openposed = ["./build/examples/openpose/openpose.bin", \
                "--image_dir", os.path.join(bound_and_crop_frames_path, str(cur_video_num)), \
                "--number_people_max", "1", \
                "--write_images", os.path.join(bound_and_crop_videos_path, str(cur_video_num)), \
                "--write_json", os.path.join(bound_and_crop_keypoints_path, str(cur_video_num)), \
                "--display", "0", \
                "--keypoint_scale", "3", \
                "--model_pose", "BODY_25"]
  openpoze = subprocess.run(openposed, stderr=subprocess.PIPE, stdout=subprocess.PIPE, cwd="/content/openpose")
  print(openpoze.stderr)

  video_maker = ["ffmpeg", "-framerate", "1", "-pattern_type", "glob", "-i", os.path.join(bound_and_crop_videos_path, str(cur_video_num), "*.jpg"), "-c:v", "libx264", "-pix_fmt", "yuv420p", os.path.join(bound_and_crop_videos_path, str(cur_video_num), "*.mp4")]
  subprocess.run(video_maker)

In [21]:
kpt_path = os.path.join(bound_and_crop_frames_path, str(cur_video_num))
final_kpt_path = os.path.join(final_keypoints_path, str(cur_video_num))
if stored == False:
  shutil.copytree(temp_kpt_path, final_kpt_path)
  stored = True

Repeat for all videos!